In [ ]:
'''
save Polynomial Cumulative for plotting
polynomial regression using run time (days)

Cell Line 2:
    in the case of use_feed_conc, use_conc_after_feed = True, False
    check feed concentration species
    also CL1 and 3

Rolling Regression: add "run time middle" & "Conc Mid" for each speceis
'''

# Test

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 200)
from CCDPApy.helper import input_path, output_path
from CCDPApy import FedBatchParameters, FedBatchCellCulture, FedBatchCellLine, FedBatchExpriment
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets
import seaborn as sns

culture_handler = FedBatchCellCulture()
culture_handler.load_data(file=input_path('fed_batch_data.xlsx'))
culture_data = culture_handler.get_pre_process_data()

cl_name, cl_id = 'CL2', '21'
use_feed_conc, use_conc_after_feed = False, True

cell_line_handler = FedBatchCellLine(
    cell_line_name=cl_name, 
    data=culture_data)
cell_line_data = cell_line_handler.get_pre_process_data()

exp_handler = FedBatchExpriment(
    cell_line_name=cl_name, 
    cell_line_id=cl_id, 
    data=cell_line_data,
    use_feed_conc=use_feed_conc,
    use_conc_after_feed=use_conc_after_feed)

exp_handler.in_process()
exp_handler.polynomial()
exp_handler.rolling_window_polynomial(degree=3, window=6)

In [13]:
# Get data
df1 = exp_handler.conc
df2 = exp_handler.cumulative_conc
df3 = exp_handler.sp_rate

# Options
spc_list = list(exp_handler.get_species().keys())
spc_list.remove('cell')
spc_list.remove('product')
spc_list = [(s.upper(), s.capitalize()) for s in spc_list]
date_list = ['Run Time (day)', 'Run Time (hr)']
profile_list = [
    ('Concentration', 1), 
    ('Cumulative Production/Consumption', 2), 
    ('SP. Rate', 3), 
    ('All Profiles', 4)]
method_list = df3['method'].unique()

df = {1: df1, 2: df2, 3: df3}
titles = {1: 'Concentration Profile', 2: 'Cumulative Profile', 3: 'SP. Rate Profile'}

sns.set_style("whitegrid", {'grid.linestyle': ':'})

spc_btn = widgets.Dropdown(
    options=spc_list, 
    description="Species")

spc_btn = widgets.SelectMultiple(
    options=spc_list,
    # value=[spc_list[0][1]],
    rows=3,
    description='Species',
    disabled=False
)

profile_btn = widgets.ToggleButtons(
    options=profile_list,
    description='Profiles:',
    disabled=False,
    button_style=''
)

date_btn = widgets.ToggleButtons(
    options=date_list, 
    description="Run Time:")

height_bar = widgets.IntSlider(
    value=3,
    min=1,
    max=20,
    step=1,
    description='Height:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

width_bar = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Width:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def save_fig(btn):
    path = file_path_input.value.strip()
    fig.savefig(path)
    print(f"Plot exported to {path}")

file_path_input = widgets.Text(
    placeholder="Enter file path",
    value=output_path('fig.png'),
    description='File Path:',
    disabled=False)

export_btn = widgets.Button(description="Export Graph")
export_btn.on_click(save_fig)

def plot_profile(spc, profiles, date, height, width):
    n_spc = len(spc)
    fig_size = (width*3, height*n_spc)

    if n_spc!=0:

        if profiles==4:
            # Create multiple subplots
            fig, axes = plt.subplots(n_spc, 3, figsize=fig_size, tight_layout=True, squeeze=False)
            for i, s in enumerate(spc):
                # Concentration
                ax = axes[i, 0]
                mask = df[1]['species']==s
                sns.lineplot(ax=ax, data=df1[mask], x=date, y='value', estimator=None, marker='o')
                ax.set_title(f"{s} {titles[1]}")
                ax.set_ylabel(f"{s} {df1[mask]['unit'].values[0]}")

                # Cumulative
                ax = axes[i, 1]
                mask = df[2]['species']==s
                data = df2[mask]
                # two-point
                data1 = data[data['method']=='twoPoint']
                sns.scatterplot(ax=ax, data=data1, x=date, y='value', estimator=None)
                # polynomial
                if 'polynomial' in data['method'].unique():
                    data2 = data[data['method']=='polynomial']
                    sns.lineplot(ax=ax, data=data2, x=date, y='value', estimator=None)

                ax.set_title(f"{s} {titles[2]}")
                ax.set_ylabel(f"{s} {df2[mask]['state'].values[0]} {df2[mask]['unit'].values[0]}")
                ax.legend(data['method'].unique())

                # SP. rate
                ax = axes[i, 2]
                mask = df[3]['species']==s
                sns.lineplot(ax=ax, data=df3[mask], x=date, y='value', estimator=None, hue='method', style='method', marker='o')
                ax.set_title(f"{s} {titles[3]}")
                ax.set_ylabel(f"{s} {df3[mask]['unit'].values[0]}")
            
        else:
            # Create plots
            fig, axes = plt.subplots(n_spc, 1, figsize=(width, height*n_spc), tight_layout=True, squeeze=False)
            axes = axes.flatten()
            df_profile = df[profiles]
            for i, s in enumerate(spc):
                ax = axes[i]
                data = df_profile[df_profile['species']==s]
                ylabel = f"{s} {data['unit'].values[0]}"
                if profiles==1: # Concentration
                    sns.lineplot(ax=ax, data=data, x=date, y='value', estimator=None, marker='o')
                elif profiles==2:
                    # two-point
                    data1 = data[data['method']=='twoPoint']
                    sns.scatterplot(ax=ax, data=data1, x=date, y='value', estimator=None)
                    # polynomial
                    if 'polynomial' in data['method'].unique():
                        data2 = data[data['method']=='polynomial']
                        sns.lineplot(ax=ax, data=data2, x=date, y='value', estimator=None)
                    ax.legend(data['method'].unique())
                    ylabel = f"{s} {data['state'].values[0]} {data['unit'].values[0]}"

                elif profiles==3:
                    sns.lineplot(ax=ax, data=data, x=date, y='value', estimator=None, hue='method', style='method', marker='o')
            
                # layout
                ax.set_ylabel(ylabel)
                ax.set_title(f"{s} {titles[profiles]}")
        #plt.savefig('/Users/galileo/Desktop/test.png')
        plt.show()

interactive_plot = interactive(
    plot_profile, 
    spc=spc_btn, 
    profiles=profile_btn,
    date=date_btn, 
    height=height_bar, 
    width=width_bar)

export_box = widgets.HBox([export_btn, file_path_input])
# plot = widgets.VBox([export_box, interactive_plot])

interactive_plot

interactive(children=(SelectMultiple(description='Species', options=(('ALANINE', 'Alanine'), ('ARGININE', 'Arg…

In [3]:
df = exp_handler.cumulative_conc
mask = (df['species']=='Alanine') & (df['method']=='polynomial')
df[mask]

,Run Time (day),Run Time (hr),value,unit,state,method,species,ID,degree
425,0,0.000000,-0.024572,(mmol),Consumed,polynomial,Alanine,11,3.0
426,0,2.358249,-0.016124,(mmol),Consumed,polynomial,Alanine,11,3.0
427,0,4.716498,-0.008585,(mmol),Consumed,polynomial,Alanine,11,3.0
428,0,7.074747,-0.001935,(mmol),Consumed,polynomial,Alanine,11,3.0
429,0,9.432997,0.003844,(mmol),Consumed,polynomial,Alanine,11,3.0
...,...,...,...,...,...,...,...,...,...
520,9,224.033670,-0.545403,(mmol),Consumed,polynomial,Alanine,11,3.0
521,9,226.391919,-0.535065,(mmol),Consumed,polynomial,Alanine,11,3.0
522,9,228.750168,-0.523757,(mmol),Consumed,polynomial,Alanine,11,3.0
523,9,231.108418,-0.511461,(mmol),Consumed,polynomial,Alanine,11,3.0


In [ ]:
exp_handler.get_processed_data()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets
import seaborn as sns

# from CCDPApy.cell_line import bioprocess_pipeline
from CCDPApy import FedBatchParameters, FedBatchCellCulture, FedBatchCellLine, FedBatchExpriment
from CCDPApy.helper import input_path, output_path

pd.set_option('display.max_columns', 200)

from CCDPApy.helper import input_path, split_df

In [ ]:
path = input_path('fed_batch_data2.xlsx')
sheets = pd.read_excel(path, sheet_name=None)
sheets.keys()

In [ ]:
param_1 = FedBatchParameters(cell_line_name='CL1',
                             use_concentration_after_feed=False,
                             use_feed_concentration=True,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

param_2 = FedBatchParameters(cell_line_name='CL2',
                             use_concentration_after_feed=True,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

param_3 = FedBatchParameters(cell_line_name='CL3',
                             use_concentration_after_feed=False,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

path = input_path('fed_batch_data.xlsx')
cl_handler = FedBatchCellCulture()
cl_handler.load_data(file=path)

data_set = cl_handler.get_all_data()
cl_line_handler = FedBatchCellLine(cell_line_name='CL1', 
                                              data=data_set,
                                              use_feed_conc=True,
                                              use_conc_after_feed=False)
data_filtered = cl_line_handler.get_all_data()
exp_handler = FedBatchExpriment(cell_line_name='CL1', 
                                        cell_line_id='11', 
                                        data=data_filtered,
                                        use_feed_conc=True,
                                        use_conc_after_feed=False)
exp_handler.in_process()
exp_handler.polynomial()
exp_handler.rolling_window_polynomial(degree=3, window=6)

In [5]:
# Get data
df1 = exp_handler.conc
df2 = exp_handler.cumulative_conc
df3 = exp_handler.sp_rate

# Options
spc_list = list(exp_handler.get_species().keys())
spc_list.remove('cell')
spc_list.remove('product')
spc_list = [(s.upper(), s.capitalize()) for s in spc_list]
date_list = ['Run Time (day)', 'Run Time (hr)']
profile_list = [
    ('Concentration', 1), 
    ('Cumulative Production/Consumption', 2), 
    ('SP. Rate', 3), 
    ('All Profiles', 4)]
method_list = df3['method'].unique()

df = {1: df1, 2: df2, 3: df3}
titles = {1: 'Concentration Profile', 2: 'Cumulative Profile', 3: 'SP. Rate Profile'}

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets
import seaborn as sns

spc_btn = widgets.Dropdown(
    options=spc_list, 
    description="Species")

spc_btn = widgets.SelectMultiple(
    options=spc_list,
    # value=[spc_list[0][1]],
    rows=3,
    description='Species',
    disabled=False
)

profile_btn = widgets.ToggleButtons(
    options=profile_list,
    description='Profiles:',
    disabled=False,
    button_style=''
)

date_btn = widgets.ToggleButtons(
    options=date_list, 
    description="Run Time:")

height_bar = widgets.IntSlider(
    value=3,
    min=1,
    max=20,
    step=1,
    description='Height:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

width_bar = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Width:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def save_fig(btn):
    path = file_path_input.value.strip()
    fig.savefig(path)
    print(f"Plot exported to {path}")

file_path_input = widgets.Text(
    placeholder="Enter file path",
    value=output_path('fig.png'),
    description='File Path:',
    disabled=False)

export_btn = widgets.Button(description="Export Graph")
export_btn.on_click(save_fig)

def plot_profile(spc, profiles, date, height, width):
    n_spc = len(spc)
    fig_size = (width*3, height*n_spc)

    if n_spc!=0:

        if profiles==4:
            # Create multiple subplots
            fig, axes = plt.subplots(n_spc, 3, figsize=fig_size, tight_layout=True, squeeze=False)
            for i, s in enumerate(spc):
                # Concentration
                ax = axes[i, 0]
                mask = df[1]['species']==s
                sns.lineplot(ax=ax, data=df1[mask], x=date, y='value', estimator=None)
                ax.set_title(f"{s} {titles[1]}")
                ax.set_ylabel(f"{s} {df1[mask]['unit'].values[0]}")

                # Cumulative
                ax = axes[i, 1]
                mask = df[2]['species']==s
                data = df2[mask]
                # two-point
                data1 = data[data['method']=='twoPoint']
                sns.scatterplot(ax=ax, data=data1, x=date, y='value', estimator=None)
                # polynomial
                if 'polynomial' in data['method'].unique():
                    data2 = data[data['method']=='polynomial']
                    sns.lineplot(ax=ax, data=data2, x=date, y='value', estimator=None)

                ax.set_title(f"{s} {titles[2]}")
                ax.set_ylabel(f"{s} {df2[mask]['unit'].values[0]}")
                ax.legend(data['method'].unique())

                # SP. rate
                ax = axes[i, 2]
                mask = df[3]['species']==s
                sns.lineplot(ax=ax, data=df3[mask], x=date, y='value', estimator=None, hue='method', style='method')
                ax.set_title(f"{s} {titles[3]}")
                ax.set_ylabel(f"{s} {df3[mask]['unit'].values[0]}")
            
        else:
            # Create plots
            fig, axes = plt.subplots(n_spc, 1, figsize=(width, height*n_spc), tight_layout=True, squeeze=False)
            axes = axes.flatten()
            df_profile = df[profiles]
            for i, s in enumerate(spc):
                ax = axes[i]
                data = df_profile[df_profile['species']==s]
                if profiles==1:
                    sns.lineplot(ax=ax, data=data, x=date, y='value', estimator=None)
                elif profiles==2:
                    # two-point
                    data1 = data[data['method']=='twoPoint']
                    sns.scatterplot(ax=ax, data=data1, x=date, y='value', estimator=None)
                    # polynomial
                    if 'polynomial' in data['method'].unique():
                        data2 = data[data['method']=='polynomial']
                        sns.lineplot(ax=ax, data=data2, x=date, y='value', estimator=None)
                    ax.legend(data['method'].unique())

                elif profiles==3:
                    sns.lineplot(ax=ax, data=data, x=date, y='value', estimator=None, hue='method', style='method')
            
                # layout
                ax.set_ylabel(f"{s} {data['unit'].values[0]}")
                ax.set_title(f"{s} {titles[profiles]}")
        #plt.savefig('/Users/galileo/Desktop/test.png')
        plt.show()

interactive_plot = interactive(
    plot_profile, 
    spc=spc_btn, 
    profiles=profile_btn,
    date=date_btn, 
    height=height_bar, 
    width=width_bar)

export_box = widgets.HBox([export_btn, file_path_input])
# plot = widgets.VBox([export_box, interactive_plot])

interactive_plot

interactive(children=(SelectMultiple(description='Species', options=(('ALANINE', 'Alanine'), ('ARGININE', 'Arg…

In [ ]:
plt.savefig('/Users/galileo/Desktop/b.png')

In [ ]:
output_path('fig.png')

In [ ]:
def save_fig(btn):
    interactive_plot
    # plt.savefig("exported_plot.png")

    path = file_path_input.value.strip()
    print(f"Plot exported to {path}")

file_path_input = widgets.Text(
    placeholder="Enter file path",
    value=output_path('fig.png'),
    description='File Path:',
    disabled=False)

export_btn = widgets.Button(description="Export Graph")
export_btn.on_click(save_fig)

export_box = widgets.HBox([file_path_input, export_btn])

export_box

In [ ]:
if n_spc!=0
    if profiles==4:
        # Create multiple subplots
        fig, axes = plt.subplots(n_spc, 3, figsize=fig_size, tight_layout=True)
        ax = axes.flatten()

        # Concentration
        mask = df[1]['species']==spc
        sns.lineplot(ax=ax[0], data=df1[mask], x=date, y='value', estimator=None)
        ax[0].set_title(titles[1])
        ax[0].set_ylabel(f"{spc} {df1[mask]['unit'].values[0]}")

        # Cumulative
        mask = df[2]['species']==spc
        data = df2[mask]
        # two-point
        data1 = data[data['method']=='twoPoint']
        sns.scatterplot(ax=ax[1], data=data1, x=date, y='value', estimator=None)
        # polynomial
        if 'polynomial' in data['method'].unique():
            data2 = data[data['method']=='polynomial']
            sns.lineplot(ax=ax[1], data=data2, x=date, y='value', estimator=None)

        ax[1].set_title(titles[2])
        ax[1].set_ylabel(f"{spc} {df2[mask]['unit'].values[0]}")
        ax[1].legend(data['method'].unique())
        
        # SP. rate
        mask = df[3]['species']==spc
        sns.lineplot(ax=ax[2], data=df3[mask], x=date, y='value', estimator=None, hue='method', style='method')
        ax[2].set_title(titles[3])
        ax[2].set_ylabel(f"{spc} {df3[mask]['unit'].values[0]}")
            


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5), squeeze=False)
axes.flatten()

In [ ]:
msk = df3['species']=='Glucose'
fig, ax = plt.subplots(figsize=(5, 3))
sns.lineplot(ax=ax, data=df3[msk], x='Run Time (hr)', y='value', estimator=None,
hue='method', style='method')
plt.show()

In [ ]:
df3[msk]['unit'].values[0]

In [ ]:
df3.head()

In [ ]:
widgets.ToggleButtons(
    options=profile_list,
    description='Profiles:',
    disabled=False,
    button_style=''
)

In [ ]:
widgets.SelectMultiple(
    options=spc_list,
    #value=['Oranges'],
    #rows=10,
    description='Fruits',
    disabled=False
)

In [ ]:
#cl_line_handler.in_process()
#cl_line_handler.polynomial()

In [ ]:
exp_handler.get_species('all').keys()

In [ ]:
cell = exp_handler.get_species('cell')
igg = exp_handler.get_species('product')
spc = exp_handler.get_species('glucose')

In [ ]:
cell.cumulative_conc

In [ ]:
spc.sp_rate_rolling

In [ ]:
spc.sp_rate_poly

In [ ]:
exp_handler.in_process()
exp_handler.polynomial()


In [ ]:
exp_handler.get_conc_before_feed()

In [ ]:
s = exp_handler.get_species('glucose')
c = s.cumulative_conc['value'].values
t = s.run_time_hour

In [ ]:
s.conc

In [ ]:
s.growth_rate

In [ ]:
s.measurement_index

In [ ]:
s.conc_before_feed

In [ ]:
s.sp_rate

In [ ]:
s.volume_before_sampling

In [ ]:
from CCDPApy.helper import create_col_indices, split_name_unit

In [ ]:
exp_handler.get_conc_after_feed().shape

In [ ]:
a = exp_handler.get_measured_cumulative_conc()
create_col_indices(a)

In [ ]:
a.columns

In [ ]:
column_indices = {}
for i, col in enumerate(a.columns):
    name, unit = split_name_unit(col)
    print(i, name)
    column_indices[name] = {'index': i, 'unit': unit}

In [ ]:
a.shape

In [ ]:
a.head()

In [ ]:
a.iloc[:, 28]

In [ ]:
exp_handler.cumulative_cell_production

In [ ]:
exp_handler.polynomial()

In [ ]:
exp_handler.integral_viable_cell_conc

In [ ]:
exp_handler.growth_rate

In [ ]:
exp_handler.cumulative_conc

In [ ]:
spc = exp_handler.get_species('glutamine')
spc.sp_rate

In [ ]:
exp_handler.conc

In [ ]:
exp_handler.cumulative_conc

In [ ]:
exp_handler.sp_rate

In [ ]:
spc.polynomial(deg=3)

In [ ]:
data1 = spc.cumulative_conc_poly
data2 = spc.cumulative_conc
sns.lineplot(data=data1, x='Run Time (day)', y='value')
sns.scatterplot(data=data2, x='Run Time (day)', y='value')
plt.show()

In [ ]:
spc.sp_rate_poly['value']

In [ ]:
idx = spc.measurement_index
t = spc.run_time_hour[idx]
s = spc.cumulative_conc['value'].values[idx]
deg = 4
poly_func = np.poly1d(np.polyfit(x=t, y=s, deg=deg))
poly_func

In [ ]:
poly_deriv = poly_func.deriv()
y = poly_deriv(spc.run_time_hour)
xv = spc.viable_cell_conc['value'].values
v = spc.volume_before_sampling
y / (xv * v) * 1000

In [ ]:

0.044170892
0.033420916
0.024387564
0.018535344
0.012786592
0.007218002
0.005092897
0.002941554
0.001030772
-0.000441458
-0.00074635
0.000949806
0.004161749
0.012702173

In [ ]:
poly_func(spc.run_time_hour)

In [ ]:
spc.cumulative_conc_poly['value']

In [ ]:
spc.sp_rate_poly['value']

In [ ]:

0.044170892
0.033420916
0.024387564
0.018535344
0.012786592
0.007218002
0.005092897
0.002941554
0.001030772
-0.000441458
-0.00074635
0.000949806
0.004161749
0.012702173

In [ ]:
exp_handler.get_measured_data().head()

In [ ]:
cell = exp_handler.get_species(species='cell')
cell.in_process()

In [ ]:
cell.sp_rate

In [ ]:
igg = exp_handler.get_species(species='product')
igg.in_process()
igg.sp_rate

In [ ]:
glt = exp_handler.get_species(species='glucose')
glt.in_process(True, False)

In [ ]:
glt.sp_rate

# MeasuredData Class Test

## Cell Line 1

In [ ]:
from CCDPApy.MeasuredData.Fed_batch.MeasuredData import MeasuredData
from CCDPApy.helper.helper import input_path

# Read all sheets into a dictionary
file_path = input_path('CL1.xlsx')
sheets_dict = pd.read_excel(file_path, sheet_name=None)

for sheet_name, df in sheets_dict.items():
    md = MeasuredData(data=df)

In [ ]:
from CCDPApy.helper import create_col_indices
create_col_indices(md.separate_feed_df)

In [ ]:
md.separate_feed_df.iloc[:, 0]

In [ ]:


a = create_col_indices(md.conc_before_feed_df)
b = create_col_indices(md.conc_after_feed_df)
c = create_col_indices(md.feed_conc_df)
d = create_col_indices(md.measured_cumulative_conc_df)

In [ ]:
a.keys() == b.keys() and a.keys() == c.keys() and a.keys() == d.keys()


In [ ]:
from CCDPApy.MeasuredData.MeasuredData import MeasuredData

f1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
for file in f1:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data().head()

## Cell Line 2

In [ ]:
from CCDPApy.MeasuredData.MeasuredData import MeasuredData
f2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
for file in f2:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data()

## Cell Line 3

In [ ]:
f3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']

for file in f3:
    print(f'Tesinting MeasuredData Class for {file}.')
    md = MeasuredData(file_name=file, 
                      measurement_sheet='Measured Data',
                      feed_sheet='Separate Feed Info')
    md.run_time()
    print("Complete run_time() method.")
    md.culture_volume()
    print("Complete culture_volume() method.")
    md.get_pre_data()

In [ ]:
md.get_pre_data()

# BioProcess Class Test

## In-Process

In in-process, the cumulative concentration will be calculated, <br>
and then specific rate will be calculated using the two-point calculation.

### Cell Line 1

There are the measurements of the feed concentration <br>
There are NOT measuremeents of the concentraion after feeding

In [ ]:
from CCDPApy.cell_line.Fed_batch import BioProcess

In [ ]:
from CCDPApy.cell_line.Fed_batch import BioProcess

files = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
#spc = ['Glucose','Lactate','Glutamine','Asparagine','Aspartate']

feed_concentration = True
concentration_after_feed = False

cell_line_1 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file,
                             #spc_list=spc,
                             #new_spc_list=['new'],
                             )
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_1[input_file.split('.')[0]] = bio_process
    
cell_line_1.keys()
exp_list = ['CL1_1', 'CL1_2', 'CL1_3']
exp = exp_list[0]

Display Experiment Information

In [ ]:
cell_line_1[exp].disp_data(exp_info=True)

Data about Cell

In [ ]:
cell = cell_line_1[exp].get_species('cell')
# DataFrame
cell.get_in_process.head()
# Display
# cell.disp_in_process

In [ ]:
cell.

Data about Oxygen

In [ ]:
oxy = cell_line_1[exp].get_species('oxygen')
# DataFrame
oxy.get_in_process
# Display
# oxy.disp_in_process

Data about Product(IgG)

In [ ]:
product = cell_line_1[exp].get_species('product')
# DataFrame
product.get_in_process
# Display
# product.disp_in_process

Data about other metabolite

In [ ]:
glc = cell_line_1[exp].get_species('glucose')
glc.get_in_process

In-process data

In [ ]:
bio_process.get_pre_process()

### Cell Line 2

In [ ]:
from CCDPApy import BioProcess

files = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']

feed_concentration = False
concentration_after_feed = True

cell_line_2 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file)
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_2[input_file.split('.')[0]] = bio_process

In [ ]:
cell = bio_process.get_species('cell')
cell.get_in_process

In [ ]:
igg = bio_process.get_species('product')
igg.get_in_process

In [ ]:
ala = bio_process.get_species('alanine')
ala.get_in_process

### Cell Line 3

In [ ]:
from CCDPApy import BioProcess

files = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']

feed_concentration = False
concentration_after_feed = False

cell_line_3 = {}
for input_file in files:
    print(f'Tesinting BioProcess Class for {input_file}.')
    bio_process = BioProcess(file_name=input_file)
    bio_process.in_process(feed_concentration=feed_concentration, 
                           concentration_after_feed=concentration_after_feed)
    print(f'Complete In-Process.')
    cell_line_3[input_file.split('.')[0]] = bio_process

## Display Pre-Process Data

In [ ]:
#bio_process.disp_data(exp_info=False, process='prepro')

## Post-Process

### Polynomial Regression

Cell Line 1

In [ ]:
cell_line_1

In [ ]:
poly_file = 'polynomial_order_1.xlsx'

for name, cl in cell_line_1.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

In [ ]:
cell = cell_line_1['CL1_1'].get_species('cell')
cell.get_post_process

Cell Line 2

In [ ]:
cell_line_2

In [ ]:
poly_file = 'polynomial_order_2.xlsx'

for name, cl in cell_line_2.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

In [ ]:
ala = cell_line_2['CL2_1'].get_species('alanine')
ala.get_in_process_data

Cell Line 3

In [ ]:
cell_line_3

In [ ]:
poly_file = 'polynomial_order_3.xlsx'

for name, cl in cell_line_3.items():
    print(f'polynomial regression method. {name}.')
    cl.poly_regression(polyorder_file=poly_file)
    print(f'Completed.')

### Rolling Window Polynomial Regression

Cell Line 1

In [ ]:
for name, cl in cell_line_1.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

In [ ]:
cl1_1 = cell_line_1['CL1_1']
cl1_1

In [ ]:
cl1_1.get_post_process_data()

In [ ]:
cell = cl1_1.get_species('cell')
cell.get_post_process_logistic

In [ ]:
bio_process.get_post_process_data()

In [ ]:
cl1_1 = cell_line_1['CL1_1']
ala = cl1_1.get_species('alanine')
ala._sp_rate_rollpoly, ala._t_mid

Cell Line 2

In [ ]:
for name, cl in cell_line_2.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

Cell Line 3

In [ ]:
for name, cl in cell_line_3.items():
    print(f'rolling polynomial regression method. {name}.')
    cl.roll_regression(degree=3, windows=6)
    print(f'Completed.')

# Interactive Plot

In [ ]:
from CCDPApy import bioprocess_pipeline

key_cl1 = {'use_feed_conc': False,
           'use_conc_after_feed': False,}
cell_line = bioprocess_pipeline(cell_culture_type='fed-batch',
                                input_file_name='CL1.xlsx',
                                **key_cl1)
cell_line.interactive_plot()

In [ ]:
from CCDPApy import CellLine    # used to aggregate all cell lines and experiments data
from CCDPApy import bioprocess_pipeline
from CCDPApy.BioProcess.BioProcess import BioProcess

input_files_1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
input_files_2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
input_files_3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']
measurement_sheet = 'Measured Data'
feed_sheet = 'Separate Feed Info'
poly_file = 'polynomial_order_1.xlsx'

cell_line = CellLine()

# Cell Line 1
for input in input_files_1:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=True, concentration_after_feed=False)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)
# Cell Line 2
for input in input_files_2:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=False, concentration_after_feed=True)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)
# Cell Line 3
for input in input_files_3:
    bp = BioProcess(file_name=input,)
    bp.in_process(feed_concentration=False, concentration_after_feed=False)
    bp.polynomial(polyorder_file=poly_file)
    bp.roll_regression(degree=3, windows=6)
    cell_line.add_bio_process(bio_process=bp)

cell_line.disp_cell_lines()

cell_line.interactive_plot(mode='inline')

## Growth rate

In [ ]:
import plotly.express as px
df = cell_line.get_plot_data()
cell_df = df[(df['species']=='cell')]
df1 = cell_df[(cell_df['profile']=='concentration') & (cell_df['kind']!='viability')]
df2 = cell_df[cell_df['kind']=='viability']

fig1 = px.line(df1, x='runTime', y='value', title='Cell concentration', markers=True,
              color='kind', line_dash='cellLine', symbol='runID')
fig1.update_layout(yaxis_title='(10^6 cells/mL)')

df2 = df2.rename(columns={'value': 'viability'})
fig2 = px.line(df2, x='runTime', y='viability', markers=True,
               color='cellLine', line_dash='cellLine', symbol='runID',
               color_discrete_sequence=px.colors.qualitative.Pastel1,)

fig2.update_traces(yaxis='y2')
for fig_data in fig2.data:
    fig1.add_trace(fig_data)

fig1.update_layout(legend_x=1.1, legend_y=1)
fig1.update_layout(yaxis2={'side': 'right', 'title': 'viability (%)', 'overlaying': "y",})
fig1

In [ ]:
df = cell_df[cell_df['profile']=='cumulative']
df1 = df[df['method']=='twoPoint']
df2 = df[df['method']=='polynomial']
fig1 = px.line(df1[df1['kind']=='integral'], x='runTime', y='value', 
                  title='Cell Cumulative concentration',
                  color='cellLine', line_dash='runID', symbol='cellLine')

fig2 = px.line(df1[df1['kind']=='cumulative'], x='runTime', y='value',
               color='cellLine', line_dash='runID', symbol='cellLine',
               color_discrete_sequence=px.colors.qualitative.Pastel1,)
fig2.update_traces(yaxis='y2')

for fig_data in fig2.data:
    fig1.add_trace(fig_data)
fig1.update_layout(legend_x=1.1, legend_y=1)
fig1.update_layout(yaxis2={'side': 'right', 'title': 'cumulative', 'overlaying': "y",})
fig1

In [ ]:
df = cell_df[(cell_df['profile']=='spRate') & (cell_df['kind']=='growthRate')]
fig = px.line(df, x='runTime', y='value', 
              title='Specific Growth Rate',
              color='cellLine', line_dash='runID', symbol='method')
fig